In [ ]:
import data_prep
from loss import dice_coef_loss, dice_coef
import evaluate
import my_utils

import os
import math
import numpy as np
import pandas as pd

from sklearn.utils import shuffle

import sys
sys.path.append('/data/ys/hh/my_project/')
from segmentation_models.backbones import get_preprocessing
from segmentation_models import Unet

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras.models import Model, load_model

# os.environ["CUDA_VISIBLE_DEVICES"]='0,1'

np.random.seed(42)
SEED = 42

ROW, COL = 512, 512
NB_CLASS = 2

BATCH_SIZE = 16
EPOCHS = 400

val_fold = 4
tst_fold = 5

sfile = 'B_DenseNet_imgsize[{},{}]_batch[{}]_epochs[{}]_valfold[{}]_tstfold[{}]_dice.h5'.format(ROW,COL, 
                                                                                                BATCH_SIZE, EPOCHS, 
                                                                                                val_fold, tst_fold)
SAVE_DIR = '/data/ys/hh/my_project/result/' + sfile[:-3]
try:
    os.mkdir(SAVE_DIR)
except:
    pass

%load_ext autoreload
%autoreload 2

In [ ]:
# load data

dirname = '/data/ys/hh/QCA_imgs/raw/Fold'
X = list(data_prep.load_data_5fold(dirname, 'X.npy', val_fold, tst_fold))
Y = list(data_prep.load_data_5fold(dirname, 'Y.npy', val_fold, tst_fold))

In [ ]:
# prep data

X, Y = data_prep.shuffle_resize_encode_data(X, Y, seed=SEED, row=ROW, col=COL)

In [ ]:
try:
    os.mkdir(SAVE_DIR + '/X')
    os.mkdir(SAVE_DIR + '/Y')
except:
    pass

my_utils.save_imgs(SAVE_DIR + '/X', X[2][...,0])
my_utils.save_imgs(SAVE_DIR + '/Y', Y[2][...,1] * 255)

In [ ]:
# preprocess X (required when using 'segmentation_models')

for i in range(3):
    X[i] = get_preprocessing('densenet121')(X[i])

In [ ]:
trn_datagen_args = dict(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, fill_mode='nearest')
trn_generator = data_prep.augment_data(X[0], Y[0], BATCH_SIZE, SEED, trn_datagen_args)
vld_generator = data_prep.augment_data(X[1], Y[1], BATCH_SIZE, SEED)

In [ ]:
# prepare model

model = Unet(input_shape = X[0][0].shape, backbone_name='densenet121', encoder_weights='imagenet', classes=NB_CLASS)
model = my_utils.ModelMGPU(model, gpus=4)
adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])
model.summary()

In [ ]:
# train model

spath = SAVE_DIR + '/' + sfile
checkpointer = ModelCheckpoint(spath, save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=20, min_lr=0.000001, verbose=1)
early_stopper = EarlyStopping(patience=80, verbose=1)
tensorboard = TensorBoard(log_dir='./tb_logs', histogram_freq=0, write_graph=True, write_images=True)

callbacks_lst = [checkpointer, reduce_lr, early_stopper, tensorboard]

history = model.fit_generator(trn_generator, validation_data=vld_generator,
                    validation_steps=math.ceil(len(X[1]) / BATCH_SIZE), steps_per_epoch=math.ceil(len(X[0]) / BATCH_SIZE),
                    epochs=EPOCHS, callbacks=callbacks_lst, shuffle=False)

spath = SAVE_DIR + '/train_history.csv'
pd.DataFrame(history.history).to_csv(spath)

In [ ]:
# load saved model

rpath = SAVE_DIR + '/' + sfile
model = load_model(rpath, 
                   custom_objects={'dice_coef_loss': dice_coef_loss,'dice_coef': dice_coef})
model.summary()

In [ ]:
# predict

P = model.predict(X[2], verbose=1)
P = (P > 0.5).astype(np.uint8)

In [ ]:
# calculate score

spath = SAVE_DIR + '/dice.csv'
evaluate.calculate_score(Y[2], P, 'Dice', save_path=spath)
spath = SAVE_DIR + '/CLD.csv'
evaluate.calculate_score(Y[2], P, 'CLD', save_path=spath)

In [ ]:
# visulalize training history

# import matplotlib.pyplot as plt

# # accuracy
# plt.plot(history.history['dice_coef'])
# plt.plot(history.history['val_dice_coef'])
# plt.title('Model Accuracy')
# plt.ylabel('DSC')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

# # loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Dice Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

In [ ]:
# save overlapped imgs

try:
    os.mkdir(SAVE_DIR + '/P')
    os.mkdir(SAVE_DIR + '/O')
except:
    pass

Y_t = Y[2][...,1]
Y_f = np.ones(Y_t.shape, dtype=np.uint8) - Y_t
P_t = P[...,1]
P_f = np.ones(P_t.shape, dtype=np.uint8) - P_t

TP = Y_t * P_t
FP = Y_f * P_t
FN = Y_t * P_f

overlap = np.zeros((len(P), ROW, COL, 3), dtype=np.uint8)
overlap[...,0] = TP * 255
overlap[...,1] = (TP + FP) * 255 # P
overlap[...,2] = (TP + FN) * 255 # Y

my_utils.save_imgs(SAVE_DIR + '/P', P_t * 255)
my_utils.save_imgs(SAVE_DIR + '/O', overlap)